In [252]:
import torch
import json

In [349]:
index_label = {
    0: 'outside',
    1: 'nonsense',
    2: 'ileocecal',
    3: 'bbps0',
    4: 'bbps1',
    5: 'bbps2',
    6: 'bbps3',
}

In [350]:
do_fix_outside = True
do_fix_nonsense = True

In [351]:
json_pred_path = "/mnt/data4/cwy/CQCExp/Experiment/SingleTask_VGG19/pred_merged_result.json"

In [352]:
outlier_json_path = "/mnt/data4/cwy/CQCExp/Experiment/R1001_detect_SingleTask_VGG19/outliers/outlier_result.json"

In [353]:
d = json.load(open(json_pred_path, 'r'))
with open("/mnt/data4/cwy/Datasets/VideoHYL/pred_out.txt",'w') as f:
    for k, v in d.items():
        print("HYL_Clip/"+k,file=f,end=' ')
        res = []
        if v[0] == 1: res.append(4)
        if v[1] == 1: res.append(5)
        if v[2] == 1: res.append(6)
        if v[3] == 1: res.append(0)
        if v[4] == 1: res.append(1)
        if v[5] == 1: res.append(2)
        if v[6] == 1: res.append(3)
        sorted(res,reverse=False)
        for it in res:
            print(it,file=f,end=' ')
        f.write('\n')

In [354]:
gt_path = '/mnt/data4/cwy/Datasets/VideoHYL/gt_out.txt'
pred_path = '/mnt/data4/cwy/Datasets/VideoHYL/pred_out.txt'

In [355]:
def read_from_imagenet(txt_path: str):
    name_all = []
    label_all = []
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        # 0位置是图片名
        line = line.strip()
        name = line.split(' ')[0]
        name_all.append(name)
        try:
            labels = [int(x)-1 for x in line.split(' ')[1:]]
        except ValueError:
            print(line)
            raise
        label_list = [0, 0, 0, 0, 0, 0, 0]
        for label in labels:
            label_list[label] = 1
        # 把后面4个bbps标签移动到最前面
        label_list = label_list[3:] + label_list[:3]
        label_all.append(label_list)

    return name_all, torch.tensor(label_all)

In [356]:
gt_names, gt_labels = read_from_imagenet(gt_path)
pred_names, pred_labels = read_from_imagenet(pred_path)
# gt_labels与pred_labels的每一行是一个长度为7的标签向量，分别对应label_dict的每个类别

In [357]:
pred_ori=pred_labels.clone().detach()

In [358]:
outliers = list(json.load(open(outlier_json_path)).values())[0]
outliers

{'0': {'0': [[122, 124],
   [162, 162],
   [165, 165],
   [173, 173],
   [498, 506],
   [581, 619],
   [621, 622],
   [1204, 1246],
   [4143, 4143]],
  '1': [[125, 161],
   [163, 164],
   [166, 172],
   [507, 510],
   [578, 580],
   [620, 620],
   [623, 623],
   [722, 722],
   [1202, 1203],
   [1247, 1247],
   [1837, 1838],
   [3949, 3949],
   [4022, 4029],
   [4142, 4142],
   [4144, 4144],
   [5037, 5039]]},
 '1': {'0': [[163, 164],
   [571, 571],
   [574, 574],
   [601, 602],
   [611, 611],
   [629, 631],
   [665, 667],
   [678, 679],
   [699, 700],
   [709, 709],
   [711, 711],
   [713, 713],
   [719, 720],
   [722, 722],
   [731, 731],
   [734, 734],
   [736, 737],
   [741, 741],
   [767, 768],
   [794, 794],
   [901, 902],
   [962, 963],
   [996, 998],
   [1006, 1006],
   [1009, 1009],
   [1051, 1053],
   [1067, 1067],
   [1069, 1069],
   [1075, 1077],
   [1079, 1079],
   [1105, 1105],
   [1149, 1151],
   [1159, 1159],
   [1171, 1172],
   [1174, 1175],
   [1231, 1231],
   [1241, 1

In [359]:
outside_outliers_101 = outliers['0']['0']
outside_outliers_010 = outliers['0']['1']
nonsense_outliers_101 = outliers['1']['0']
nonsense_outliers_010 = outliers['1']['1']

In [360]:
def merge_outside_outliers(idx_101,idx_010):
    idx_ab=[]
    idx_ab.extend(idx_101)
    idx_ab.extend(idx_010)
    idx_ab.sort(key=lambda x:x[0])
    idx_merged = [idx_ab.pop(0)]
    for it in idx_ab:
        if it[0]==idx_merged[-1][1]+1:
            idx_merged[-1][1]=it[1]
        else:
            idx_merged.append(it)
    return idx_merged 

In [361]:
outside_outliers = merge_outside_outliers(outside_outliers_101,outside_outliers_010)
outside_outliers

[[122, 173],
 [498, 510],
 [578, 623],
 [722, 722],
 [1202, 1247],
 [1837, 1838],
 [3949, 3949],
 [4022, 4029],
 [4142, 4144],
 [5037, 5039]]

In [362]:
nonsense_outliers = nonsense_outliers_101
nonsense_outliers

[[163, 164],
 [571, 571],
 [574, 574],
 [601, 602],
 [611, 611],
 [629, 631],
 [665, 667],
 [678, 679],
 [699, 700],
 [709, 709],
 [711, 711],
 [713, 713],
 [719, 720],
 [722, 722],
 [731, 731],
 [734, 734],
 [736, 737],
 [741, 741],
 [767, 768],
 [794, 794],
 [901, 902],
 [962, 963],
 [996, 998],
 [1006, 1006],
 [1009, 1009],
 [1051, 1053],
 [1067, 1067],
 [1069, 1069],
 [1075, 1077],
 [1079, 1079],
 [1105, 1105],
 [1149, 1151],
 [1159, 1159],
 [1171, 1172],
 [1174, 1175],
 [1231, 1231],
 [1241, 1242],
 [1266, 1266],
 [1269, 1269],
 [1334, 1334],
 [1347, 1347],
 [1352, 1352],
 [1354, 1354],
 [1384, 1384],
 [1386, 1386],
 [1404, 1406],
 [1499, 1499],
 [1523, 1524],
 [1584, 1584],
 [1586, 1587],
 [1721, 1721],
 [1737, 1739],
 [1741, 1741],
 [1774, 1774],
 [1784, 1784],
 [1787, 1787],
 [1841, 1842],
 [1844, 1844],
 [1941, 1941],
 [1950, 1951],
 [2041, 2041],
 [2047, 2047],
 [2056, 2056],
 [2142, 2142],
 [2377, 2377],
 [2436, 2437],
 [2549, 2549],
 [2572, 2572],
 [2585, 2587],
 [2734, 273

In [363]:
def fix_outliers_idx(pred_origin, outlier_list, val_func):
    pred_fix = pred_origin.clone().detach()
    for item in outlier_list:
        a, b = item
        for now_i in range(a,b+1):
            pred_fix[now_i] = torch.tensor(val_func(pred_origin, item, now_i))
    return pred_fix

In [364]:
def val_reverse_outside(pred,idx_item,now_i):
    a, b = idx_item
    if pred[a][0].item()==0:
        if pred[now_i][0].item()==1 or pred[now_i][1].item()==1:
            print(f'Fix {idx_item}: {now_i} not change')
            return pred[now_i].clone().detach()
        else:
            print(f'Fix {idx_item}: {now_i}=[1, 0, 0, 0, 0, 0, 0]')
            return [1, 0, 0, 0, 0, 0, 0]
    elif pred[a][0].item()==1:
        if pred[now_i][0].item()==0:
            print(f'Fix {idx_item}: {now_i} not change')
            return pred[now_i].clone().detach()
        else:
            print(f'Fix {idx_item}: {now_i}=[0, 1, 0, 0, 0, 0, 0]')
            return [0, 1, 0, 0, 0, 0, 0]
    else:
        raise ValueError

In [365]:
def val_reverse_nonsense_101(pred,idx_item,now_i):
    if pred[now_i][1].item()==0:
        print(f'Fix {idx_item}: {now_i}=[0, 1, 0, 0, 0, 0, 0]')
        return [0, 1, 0, 0, 0, 0, 0]
    elif pred[now_i][1].item()==1:
        print(f'Fix {idx_item}: {now_i} not change')
        return pred[now_i].clone().detach()
    else:
        raise ValueError

In [366]:
pred_fix = pred_labels.clone().detach()

In [367]:
if do_fix_outside: pred_fix = fix_outliers_idx(pred_fix, outside_outliers, val_reverse_outside)

Fix [122, 173]: 122 not change
Fix [122, 173]: 123 not change
Fix [122, 173]: 124 not change
Fix [122, 173]: 125 not change
Fix [122, 173]: 126 not change
Fix [122, 173]: 127 not change
Fix [122, 173]: 128 not change
Fix [122, 173]: 129 not change
Fix [122, 173]: 130 not change
Fix [122, 173]: 131 not change
Fix [122, 173]: 132 not change
Fix [122, 173]: 133 not change
Fix [122, 173]: 134 not change
Fix [122, 173]: 135 not change
Fix [122, 173]: 136 not change
Fix [122, 173]: 137 not change
Fix [122, 173]: 138 not change
Fix [122, 173]: 139 not change
Fix [122, 173]: 140 not change
Fix [122, 173]: 141 not change
Fix [122, 173]: 142 not change
Fix [122, 173]: 143 not change
Fix [122, 173]: 144 not change
Fix [122, 173]: 145 not change
Fix [122, 173]: 146 not change
Fix [122, 173]: 147 not change
Fix [122, 173]: 148 not change
Fix [122, 173]: 149 not change
Fix [122, 173]: 150 not change
Fix [122, 173]: 151 not change
Fix [122, 173]: 152 not change
Fix [122, 173]: 153 not change
Fix [122

/tmp/ipykernel_30258/1748927244.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_fix[now_i] = torch.tensor(val_func(pred_origin, item, now_i))


In [368]:
if do_fix_nonsense: pred_fix = fix_outliers_idx(pred_fix, nonsense_outliers, val_reverse_nonsense_101)

Fix [163, 164]: 163=[0, 1, 0, 0, 0, 0, 0]
Fix [163, 164]: 164=[0, 1, 0, 0, 0, 0, 0]
Fix [571, 571]: 571=[0, 1, 0, 0, 0, 0, 0]
Fix [574, 574]: 574=[0, 1, 0, 0, 0, 0, 0]
Fix [601, 602]: 601=[0, 1, 0, 0, 0, 0, 0]
Fix [601, 602]: 602=[0, 1, 0, 0, 0, 0, 0]
Fix [611, 611]: 611=[0, 1, 0, 0, 0, 0, 0]
Fix [629, 631]: 629=[0, 1, 0, 0, 0, 0, 0]
Fix [629, 631]: 630=[0, 1, 0, 0, 0, 0, 0]
Fix [629, 631]: 631=[0, 1, 0, 0, 0, 0, 0]
Fix [665, 667]: 665=[0, 1, 0, 0, 0, 0, 0]
Fix [665, 667]: 666=[0, 1, 0, 0, 0, 0, 0]
Fix [665, 667]: 667=[0, 1, 0, 0, 0, 0, 0]
Fix [678, 679]: 678=[0, 1, 0, 0, 0, 0, 0]
Fix [678, 679]: 679=[0, 1, 0, 0, 0, 0, 0]
Fix [699, 700]: 699=[0, 1, 0, 0, 0, 0, 0]
Fix [699, 700]: 700=[0, 1, 0, 0, 0, 0, 0]
Fix [709, 709]: 709=[0, 1, 0, 0, 0, 0, 0]
Fix [711, 711]: 711=[0, 1, 0, 0, 0, 0, 0]
Fix [713, 713]: 713=[0, 1, 0, 0, 0, 0, 0]
Fix [719, 720]: 719=[0, 1, 0, 0, 0, 0, 0]
Fix [719, 720]: 720=[0, 1, 0, 0, 0, 0, 0]
Fix [722, 722]: 722 not change
Fix [731, 731]: 731=[0, 1, 0, 0, 0, 0, 0]
Fix

/tmp/ipykernel_30258/1748927244.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_fix[now_i] = torch.tensor(val_func(pred_origin, item, now_i))


In [369]:
pred_labels = pred_fix.clone().detach()

In [370]:
confuse_matrix = dict()
for i in range(3):
    confuse_matrix[f'label_{index_label[i]}_TP'] = 0.
    confuse_matrix[f'label_{index_label[i]}_FP'] = 0.
    confuse_matrix[f'label_{index_label[i]}_FN'] = 0.
    confuse_matrix[f'label_{index_label[i]}_TN'] = 0.
for i in range(0, 4):  # i: predict
    for j in range(0, 4):  # j: gt
        confuse_matrix[f'label_cleansing_pred_{index_label[i + 3]}_gt_{index_label[j + 3]}'] = 0.

In [371]:
# 计算test_acc
mean_acc = float(torch.eq(pred_labels, gt_labels).float().mean().cpu())

In [372]:
in_out_labels = pred_labels[:, 0]
# 体内外标签: BoolTensor[B]
# outside时为True
label_in_out_pred = torch.gt(in_out_labels, 0)
# 体内外gt: BoolTensor[B]
label_in_out_gt = torch.gt(gt_labels[:, 0], 0)
confuse_matrix[f'label_{index_label[0]}_TP'] += float((label_in_out_pred & label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_FP'] += float((label_in_out_pred & ~label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_FN'] += float((~label_in_out_pred & label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_TN'] += float((~label_in_out_pred & ~label_in_out_gt).float().sum().cpu())


In [373]:
nonsense_logit = pred_labels[:, 1]
# 坏帧标签: BoolTensor[B]
# nonsense时为True
label_nonsense_pred = torch.gt(nonsense_logit, 0)
# 帧质量gt: BoolTensor[B]
# pred或gt是outside时不计入总数
label_nonsense_gt = torch.gt(gt_labels[:, 1], 0)
flag = ~label_in_out_pred & ~label_in_out_gt
confuse_matrix[f'label_{index_label[1]}_TP'] += float(
    (flag & label_nonsense_pred & label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_FP'] += float(
    (flag & label_nonsense_pred & ~label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_FN'] += float(
    (flag & ~label_nonsense_pred & label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_TN'] += float(
    (flag & ~label_nonsense_pred & ~label_nonsense_gt).float().sum().cpu())


In [374]:
ileo_logit = pred_labels[:, 2]
# 回盲部标签: BoolTensor[B]
label_ileo_pred = torch.gt(ileo_logit, 0)
# 回盲部gt: BoolTensor[B]
label_ileo_gt = torch.gt(gt_labels[:, 2], 0)
flag = ~label_in_out_pred & ~label_in_out_gt & ~label_nonsense_pred & ~label_nonsense_gt
confuse_matrix[f'label_{index_label[2]}_TP'] += float((flag & label_ileo_pred & label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_FP'] += float((flag & label_ileo_pred & ~label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_FN'] += float((flag & ~label_ileo_pred & label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_TN'] += float((flag & ~label_ileo_pred & ~label_ileo_gt).float().sum().cpu())


In [375]:
# 清洁度logit: FloatTensor[B, 4]
cls_logit = pred_labels[:, 3:]
# 清洁度label: IntTensor[B] (取预测值最大的，但会被outside标签抑制)
label_cls_pred = torch.argmax(cls_logit, dim=-1)
# 清洁度gt: IntTensor[B]
label_cls_gt = torch.argmax(gt_labels[:, 3:], dim=-1)
flag = ~label_in_out_pred & ~label_in_out_gt & ~label_nonsense_pred & ~label_nonsense_gt
for i in range(0, 4):  # i: predict
    for j in range(0, 4):  # j: gt
        confuse_matrix[f'label_cleansing_pred_{index_label[i + 3]}_gt_{index_label[j + 3]}'] += float((flag & torch.eq(label_cls_pred, i) & torch.eq(label_cls_gt, j)).float().sum().cpu())  # flag用于清洁度标签抑制


In [376]:
confuse_matrix

{'label_outside_TP': 511.0,
 'label_outside_FP': 3.0,
 'label_outside_FN': 6.0,
 'label_outside_TN': 4814.0,
 'label_nonsense_TP': 1388.0,
 'label_nonsense_FP': 276.0,
 'label_nonsense_FN': 52.0,
 'label_nonsense_TN': 3098.0,
 'label_ileocecal_TP': 109.0,
 'label_ileocecal_FP': 24.0,
 'label_ileocecal_FN': 15.0,
 'label_ileocecal_TN': 2950.0,
 'label_cleansing_pred_bbps0_gt_bbps0': 13.0,
 'label_cleansing_pred_bbps0_gt_bbps1': 10.0,
 'label_cleansing_pred_bbps0_gt_bbps2': 66.0,
 'label_cleansing_pred_bbps0_gt_bbps3': 9.0,
 'label_cleansing_pred_bbps1_gt_bbps0': 9.0,
 'label_cleansing_pred_bbps1_gt_bbps1': 132.0,
 'label_cleansing_pred_bbps1_gt_bbps2': 66.0,
 'label_cleansing_pred_bbps1_gt_bbps3': 5.0,
 'label_cleansing_pred_bbps2_gt_bbps0': 2.0,
 'label_cleansing_pred_bbps2_gt_bbps1': 32.0,
 'label_cleansing_pred_bbps2_gt_bbps2': 928.0,
 'label_cleansing_pred_bbps2_gt_bbps3': 127.0,
 'label_cleansing_pred_bbps3_gt_bbps0': 0.0,
 'label_cleansing_pred_bbps3_gt_bbps1': 0.0,
 'label_cleans

In [377]:
metrics = dict()

# 体内外
TP: float = confuse_matrix[f'label_{index_label[0]}_TP']
FP: float = confuse_matrix[f'label_{index_label[0]}_FP']
FN: float = confuse_matrix[f'label_{index_label[0]}_FN']
TN: float = confuse_matrix[f'label_{index_label[0]}_TN']
metrics[f'label_{index_label[0]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 帧质量
TP: float = confuse_matrix[f'label_{index_label[1]}_TP']
FP: float = confuse_matrix[f'label_{index_label[1]}_FP']
FN: float = confuse_matrix[f'label_{index_label[1]}_FN']
TN: float = confuse_matrix[f'label_{index_label[1]}_TN']
metrics[f'label_{index_label[1]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 回盲部
TP: float = confuse_matrix[f'label_{index_label[2]}_TP']
FP: float = confuse_matrix[f'label_{index_label[2]}_FP']
FN: float = confuse_matrix[f'label_{index_label[2]}_FN']
TN: float = confuse_matrix[f'label_{index_label[2]}_TN']

metrics[f'label_{index_label[2]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 四分清洁度准确率
total: float = 0.
correct: float = 0.
for i in range(3, 7):  # i: predict
    for j in range(3, 7):  # j: gt
        tmp = confuse_matrix[f'label_cleansing_pred_{index_label[i]}_gt_{index_label[j]}']
        if i == j:
            correct += tmp
        total += tmp
metrics[f'label_cleansing_acc'] = correct / total if total > 0. else 0.

In [378]:
mean_acc

0.9500508904457092

In [379]:
metrics

{'label_outside_acc': 0.9983127109111362,
 'label_nonsense_acc': 0.9318653926049024,
 'label_ileocecal_acc': 0.9874112330535829,
 'label_cleansing_acc': 0.8140735958683021}

In [286]:
cnt=0
for i in range(pred_labels.size()[0]):
    if torch.not_equal(pred_labels[i],gt_labels[i]).any():
        print(f"{i}:{pred_labels[i].numpy()} != {gt_labels[i].numpy()}")
        if (pred_labels[i][1].item()==1 and gt_labels[i][1].item()==0) or (pred_labels[i][1].item()==0 and gt_labels[i][1].item()==1):
            cnt+=1
cnt

162:[0 1 0 0 0 0 0] != [1 0 0 0 0 0 0]
165:[0 1 0 0 0 0 0] != [1 0 0 0 0 0 0]
173:[0 1 0 0 0 0 0] != [1 0 0 0 0 0 0]
497:[1 0 0 0 0 0 0] != [0 1 0 0 0 0 0]
506:[0 1 0 0 0 0 0] != [1 0 0 0 0 0 0]
510:[1 0 0 0 0 0 0] != [0 1 0 0 0 0 0]
568:[0 0 0 0 0 1 0] != [0 0 0 0 0 0 1]
571:[0 0 0 0 0 0 1] != [0 1 0 0 0 0 0]
574:[0 0 0 0 0 0 1] != [0 1 0 0 0 0 0]
575:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
578:[1 0 0 0 0 0 0] != [0 0 0 0 0 0 1]
579:[1 0 0 0 0 0 0] != [0 0 0 0 0 0 1]
580:[1 0 0 0 0 0 0] != [0 0 0 0 0 0 1]
581:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
601:[0 0 0 0 0 0 1] != [0 0 0 0 0 1 0]
602:[0 0 0 0 0 0 1] != [0 0 0 0 0 1 0]
603:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
604:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
605:[0 1 0 0 0 0 0] != [0 0 0 0 0 0 1]
611:[0 0 0 0 0 0 1] != [0 1 0 0 0 0 0]
620:[1 0 0 0 0 0 0] != [0 0 0 0 0 0 1]
623:[1 0 0 0 0 0 0] != [0 0 0 0 0 0 1]
629:[0 0 0 0 0 0 1] != [0 1 0 0 0 0 0]
630:[0 0 0 0 0 0 1] != [0 1 0 0 0 0 0]
631:[0 0 0 0 0 0 1] != [0 1 0 0 0 0 0]
635:[0 1 0 0 0 0 0] != [0

289

In [107]:
import DrawLabelColorBlock
import os
import os.path as osp
import shutil

In [108]:
if osp.exists('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred'):
    shutil.rmtree('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred')
os.makedirs('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred')

In [109]:
for i,pd in enumerate(pred_labels):
    ori_img_path = osp.join('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames',pred_names[i])
    out_img_path = osp.join('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred',pred_names[i])
    pd_np=pd.numpy()
    pd_dict={'outside': False,
           'nonsense': False,
           'ileocecal': False,
           'bbps': -1}
    if pd_np[0]==1:
        pd_dict['outside']=True
    if pd_np[1]==1:
        pd_dict['nonsense']=True
    if pd_np[2]==1:
        pd_dict['ileocecal']=True
    if pd_np[3]==1:
        pd_dict['bbps']=0
    elif pd_np[4]==1:
        pd_dict['bbps']=1
    elif pd_np[5]==1:
        pd_dict['bbps']=2
    elif pd_np[6]==1:
        pd_dict['bbps']=3
    DrawLabelColorBlock.DrawLabelColorBlockOnFrame(ori_img_path,out_img_path,**pd_dict)

In [110]:

# 把帧拼成视频
img_path = '/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/frames_with_pred/HYL_Clip/'
img_list = os.listdir(img_path)
img_list.sort(key=lambda x: int(x.split('.')[0]))
img_list

['000000.png',
 '000002.png',
 '000004.png',
 '000006.png',
 '000008.png',
 '000010.png',
 '000012.png',
 '000014.png',
 '000016.png',
 '000018.png',
 '000020.png',
 '000022.png',
 '000024.png',
 '000026.png',
 '000028.png',
 '000030.png',
 '000032.png',
 '000034.png',
 '000036.png',
 '000038.png',
 '000040.png',
 '000042.png',
 '000044.png',
 '000046.png',
 '000048.png',
 '000050.png',
 '000052.png',
 '000054.png',
 '000056.png',
 '000058.png',
 '000060.png',
 '000062.png',
 '000064.png',
 '000066.png',
 '000068.png',
 '000070.png',
 '000072.png',
 '000074.png',
 '000076.png',
 '000078.png',
 '000080.png',
 '000082.png',
 '000084.png',
 '000086.png',
 '000088.png',
 '000090.png',
 '000092.png',
 '000094.png',
 '000096.png',
 '000098.png',
 '000100.png',
 '000102.png',
 '000104.png',
 '000106.png',
 '000108.png',
 '000110.png',
 '000112.png',
 '000114.png',
 '000116.png',
 '000118.png',
 '000120.png',
 '000122.png',
 '000124.png',
 '000126.png',
 '000128.png',
 '000130.png',
 '000132.p

In [111]:
import cv2
# cv2
img = cv2.imread(osp.join(img_path, img_list[0]))
height, width, layers = img.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
os.makedirs('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/videos_with_pred', exist_ok=True)
video = cv2.VideoWriter('/mnt/data4/cwy/CQCExp/Experiment/R105_detect_fps_vitp14s336c7_40/videos_with_pred/HYL_Clip.mp4', fourcc, 12.5, (width, height))

for img_name in img_list:
    img = cv2.imread(osp.join(img_path, img_name))
    video.write(img)
    
cv2.destroyAllWindows()
video.release()



